In [4]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os
from pandas.io.json import json_normalize
import pyslha
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from collections import OrderedDict
import seaborn as sns

pd.option_context('display.max_columns', -1)

pd.options.mode.chained_assignment = None #Disable copy warnings
# plt.style.use('fivethirtyeight') #Set style
# mpl.rcParams.update({'figure.figsize' : (15,10)})  #Set general plotting options

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

In [5]:
#Merge with SModelS DataFrame
dataDF = pd.read_pickle('EWino_scanRandom.pcl')
# print(dataDF.columns.values.tolist()) #Print all columns names

#### Set r = 0 for points with no results 

In [6]:
#Set points without results with r == 0
dataDF.fillna(value={'ExptRes.result0.r' : 0.0},inplace=True)
#Sort points according to r value:
dataDF = dataDF.sort_values(by=['ExptRes.result0.r'],ascending=False)
#Add simple flag for excluded points:
dataDF['excluded'] = dataDF['ExptRes.result0.r'] > 1.

#### Add c*tau column for charginos

In [7]:
dataDF['ctau.1000024'] = 1.967e-16/dataDF['width.1000024']

### Get points excluded and allowed

In [8]:
excluded = dataDF[dataDF['excluded'] == True]
allowed = dataDF[dataDF['excluded'] == False]
print('Total number of points = %i' %len(dataDF))
print('Total excluded = %i'%(len(excluded)))
print('Total allowed = %i'%(len(allowed)))

Total number of points = 47692
Total excluded = 3389
Total allowed = 44303


In [13]:
allowed[(abs(allowed['mass.1000024']-700.) < 10.) & (allowed['mass.1000022'] < 50.)]

,filename,mass.35,mass.36,mass.37,mass.1000021,mass.1000022,mass.1000023,mass.1000024,mass.1000025,mass.1000035,...,ExptRes.result36.AnalysisSqrts (TeV),ExptRes.result36.lumi (fb-1),ExptRes.result36.dataType,ExptRes.result36.r,ExptRes.result36.r_expected,ExptRes.result36.Width (GeV),ExptRes.result36.chi2,ExptRes.result36.likelihood,excluded,ctau.1000024
12402,ew_56j5w2ky.slha,5000.0,5000.0,5000.0,9977.83666,46.482686,702.883413,703.047252,-1927.068670,1929.413310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2.822623e-13
21762,ew_qpyaxo4t.slha,5000.0,5000.0,5000.0,9899.59016,27.671633,696.412071,696.575656,-1518.492810,1522.208750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.202954e-13
31365,ew_r6rz4qkh.slha,5000.0,5000.0,5000.0,9830.07062,46.692139,691.307120,691.469991,-1228.835120,1234.883370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,4.471166e-14
39660,ew_vbsfbku3.slha,5000.0,5000.0,5000.0,9702.48540,22.714318,698.769615,698.938576,-717.307019,923.933844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,5.093800e-16
40067,ew_8i49f7pp.slha,5000.0,5000.0,5000.0,9728.45370,35.577796,704.224651,704.308657,-718.458336,990.238873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,4.839060e-16
24130,ew_4xhaa64w.slha,5000.0,5000.0,5000.0,9881.65977,43.346481,702.684828,702.359462,-707.500273,1532.002820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,4.605685e-16


In [14]:
row = allowed.loc[12402]

In [15]:
row['mass.1000024'],row['mass.1000022']

(703.047252, 46.4826862)

In [17]:
print(row['ExptRes.result0.r'],row['ExptRes.result0.AnalysisID'])

0.7417107360457573 ATLAS-SUSY-2019-08


In [21]:
print(row['xsec13TeV(fb).1000023_1000024'],row['xsec13TeV(fb).-1000024_1000023'])
print('Total = ',row['xsec13TeV(fb).1000023_1000024']+row['xsec13TeV(fb).-1000024_1000023'])

5.03585082 1.75978084
Total =  6.79563166


In [ ]:
#Cross-sections from Prospino:
# (1000023_1000024): LO = 5.64, NLO = 6.33
# (-1000024_1000023): LO = 1.88, NLO = 2.27
# Total: LO = 7.52, NLO = 8.6 (K = 1.14)

#Cross-sections from Pythia6:
# (1000023_1000024): LO = 5.958
# (-1000024_1000023): LO = 1.962
# Total: LO = 7.91

#Cross-sections for winos (LHC SUSY Xsec group):
# (1000023_1000024): NLO+NLL = 6.99
# (-1000024_1000023): NLO+NLL = 2.52
# Total: NLO+NLL = 9.51


